In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(diagnostics_port=None)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34235,Cluster Workers: 2 Cores: 2 Memory: 13.62 GB


In [ ]:
!python -m pip install dask[complete]

In [ ]:
!wget http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip --no-check-certificate

--2021-06-30 03:33:12--  http://eforexcel.com/wp/wp-content/uploads/2020/09/5m-Sales-Records.zip
Resolving eforexcel.com (eforexcel.com)... 162.214.80.24
Connecting to eforexcel.com (eforexcel.com)|162.214.80.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201708941 (192M) [application/zip]
Saving to: ‘5m-Sales-Records.zip’

5m-Sales-Records.zi 100%[===================>] 192.36M  54.0MB/s    in 4.0s    

2021-06-30 03:33:16 (48.5 MB/s) - ‘5m-Sales-Records.zip’ saved [201708941/201708941]



In [ ]:
!unzip 5m-Sales-Records.zip

Archive:  5m-Sales-Records.zip
  inflating: 5m Sales Records.csv    


In [ ]:
!ls

'5m Sales Records.csv'	 dask-worker-space   millionsongsubset.tar.gz
 5m-Sales-Records.zip	 MillionSongSubset   sample_data


In [ ]:
import pandas as pd
pd_df = pd.read_csv("5m Sales Records.csv")

In [ ]:
pd_df

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,Middle East and North Africa,Iraq,Household,Online,L,3/17/2014,940436398,4/23/2014,4884,668.27,502.54,3263830.68,2454405.36,809425.32
4999996,Europe,Monaco,Meat,Offline,H,11/8/2015,407689177,11/28/2015,3142,421.89,364.69,1325578.38,1145855.98,179722.40
4999997,Australia and Oceania,Solomon Islands,Clothes,Online,C,6/1/2020,727000367,7/18/2020,4419,109.28,35.84,482908.32,158376.96,324531.36
4999998,Australia and Oceania,Marshall Islands,Cosmetics,Offline,L,2/12/2020,714043796,3/22/2020,282,437.20,263.33,123290.40,74259.06,49031.34


In [ ]:
pd_df.Region.value_counts()

Sub-Saharan Africa                   1297687
Europe                               1296674
Asia                                  729864
Middle East and North Africa          621539
Central America and the Caribbean     540528
Australia and Oceania                 405577
North America                         108131
Name: Region, dtype: int64

In [ ]:
dict_region = {
    "Sub-saharan Africa": 1,
    "Europe":2,
    "Asia":3,
    "Middle East and North Africa":4,
    "Central America and the Caribbean":5,
    "Australia and Oceania":6,
    "North America":7
}

In [ ]:
def slow_code(region_name):
  for k,v in dict_region.items():
    if k == region_name:
      return v
  return None
slow_code("Europe")

2

In [ ]:
import dask.dataframe as dd
dd_df = dd.read_csv("5m Sales Records.csv")

In [ ]:
dd_df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93


In [ ]:
%%time
# Normal Pandas
pd_df["RegionCode"] = pd_df.Region.map(slow_code)

CPU times: user 3.2 s, sys: 10.5 ms, total: 3.21 s
Wall time: 3.22 s


In [ ]:
pd_df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,RegionCode
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25,6.0
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40,2.0
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77,7.0
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96,2.0
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93,4.0


In [ ]:
%%time
# Dask
dd_df = dd_df.assign(RegionCode = dd_df.Region.map(slow_code,meta=(None,int)))

CPU times: user 7.33 ms, sys: 780 µs, total: 8.11 ms
Wall time: 8.69 ms


In [ ]:
dd_df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,RegionCode
0,Australia and Oceania,Palau,Office Supplies,Online,H,3/6/2016,517073523,3/26/2016,2401,651.21,524.96,1563555.21,1260428.96,303126.25,6.0
1,Europe,Poland,Beverages,Online,L,4/18/2010,380507028,5/26/2010,9340,47.45,31.79,443183.00,296918.60,146264.40,2.0
2,North America,Canada,Cereal,Online,M,1/8/2015,504055583,1/31/2015,103,205.70,117.11,21187.10,12062.33,9124.77,7.0
3,Europe,Belarus,Snacks,Online,C,1/19/2014,954955518,2/27/2014,1414,152.58,97.44,215748.12,137780.16,77967.96,2.0
4,Middle East and North Africa,Oman,Cereal,Offline,H,4/26/2019,970755660,6/2/2019,7027,205.70,117.11,1445453.90,822931.97,622521.93,4.0
